In [1]:

import torch
import pandas as pd
import re
import multidict as multidict

import numpy as np

import os
import re
from PIL import Image
from os import path
from wordcloud import WordCloud
import matplotlib.pyplot as plt

import tweepy

from unicodedata import normalize

import time

from matplotlib.font_manager import FontProperties



In [2]:
def scrapeo(hashtag,cantidad=1000,fecha='2017-09-10'):
    message,retweet_count,created_at,user_name,coord,geo,metadata=[],[],[],[],[],[],[]
    i = 0
    tweepy.Status
    for tweet in tweepy.Cursor(api.search,q="#"+str(hashtag),count = cantidad, lang = 'es', since = fecha,tweet_mode='extended').items():
        if 'RT' in tweet.full_text:
            continue
        message.append(tweet.full_text)
        retweet_count.append(tweet.retweet_count)
        created_at.append(tweet.created_at)
        user_name.append(tweet.user.name)
        coord.append(tweet.coordinates)
        geo.append(tweet.geo)
        metadata.append(tweet.metadata)
        i+=1
        if i%1000 == 0: print (i,tweet.full_text)
        if i == 10000: break
    print (len(message),len(retweet_count),len(created_at),len(user_name),len(coord),len(geo))
    df=pd.DataFrame({'Message':message,
                    'Retweet Count':retweet_count,
                    'Created At':created_at,
                    'Username':user_name,
                     'Geo':geo,
                     'Coord':coord,
                     'Metadata':metadata
                    })
    df.to_csv("Twitter_%s.csv"%(hashtag))
    return(df)

    

def diccionario (msj):
    #Patrones a filtrar
    pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    
    #Hace el word to ix
    word_to_ix = {}
    word_cant = {}
    emoji = []
    emoji_ix = {}
    emoji_cant = {}
    tw_corr = []
    for i,twit in enumerate(msj):
        if i %10000 == 0: print(i, twit)
        txt = pattern.sub('',twit)
        txt = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1",normalize( "NFD", txt), 0, re.I)
        txt = normalize( 'NFC', txt).lower()
        txt = txt.replace('\n',' ').replace(';','').replace('°','').replace('ñ','n').replace('\ufe0f','').replace('+','').replace('¡','').replace('¿','').replace('?','').replace('▯','').replace('(','').replace('|','').replace('!','').replace(')','').replace('.','').replace('-','').replace('_','').replace(',','').replace('#','').replace(':',' ').replace('(',' ').replace(')',' ')
        tt = txt
        for i in txt:
            if str(i).isascii()==False:
                tt = tt.replace(i,' '+i.encode('unicode-escape').decode('ASCII')+' ')
                emoji.append(i)
        txt = tt
                
        nlist = txt.lower().split(' ')

        for word in nlist:
            
            if ('@' in word) and ('RELAMPAGO2018' not in word) and ('RELAMPAGO_edu' not in word) or ('jaj' in word):
                continue
            
            if word not in word_to_ix:
                word_to_ix[word] = len(word_to_ix)
                word_cant[word]  = 1
            else: word_cant[word]+=1
        for emo in emoji:
            if emo not in emoji_ix:
                emoji_ix[emo] = len(emoji_ix)
                emoji_cant[emo]  = 1
            else: emoji_cant[emo]+=1
                
        txt = " ".join(nlist) 
        tw_corr.append(txt)

    vocab_size = len(word_to_ix)
    num_labels = 2  
    
    return(tw_corr,word_to_ix,word_cant,emoji_ix,emoji_cant)

def makeImage(text):
    alice_mask = np.array(Image.open("cord.jpeg"))

    wc = WordCloud(background_color="white", max_words=200, mask=alice_mask)
    # generate word cloud
    wc.generate_from_frequencies(text)

    # show
    fig = plt.figure(figsize=(20,20),dpi=100)
    plt.imshow(wc, interpolation="bilinear",cmap=plt.get_cmap("coolwarm"))
    #plt.axis("off")
    plt.tick_params(bottom=False,top=False,left=False,right=False,labelbottom=False,labelleft = False)
    #plt.show()
    plt.savefig('Multiword')

def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix)+1)
    
    for word_ in sentence.split(' '):
        if word_ not in word_to_ix: 
#            print(word_ +' no esta \n')
            continue
        w = word_to_ix[word_]
        vec[w] = vec[w] or 1
        
#        print(word_ +' si esta '+ str(w))
    
        
    
    return vec.view(1, -1).numpy().astype(int)

In [3]:
tw = pd.read_csv('tweets.csv',sep = ';')
msj = tw['text']

## ESTO NO SE TOCA

In [73]:
tw_corr,word_ix,word_cant,emoji_ix,emoji_cant = diccionario(msj)


0 ¡Ni la lluvia los detuvo!🌧️ Este mañana se corrieron los #10KTernium con gran participación de corredores nicoleños. 🏅

🏃‍♂️Caballeros: 1° Ulises Otero, 2° Juan Ansio, 3° Mario Raschetti 
🏃‍♀️Damas: 1° Ludmila Martinez, 2° Nerea Mendoza, 3° María de los Ángeles Troffe https://t.co/eZceEaGQO7
10000 No tenía ni ganas de viajar, gracias lluvia😝
20000 SALUD: LA PROVINCIA BRINDÓ RECOMENDACIONES SANITARIAS ANTE LAS INTENSAS LLUVIAS: El Ministerio de Salud advierte que como consecuencia del fenómeno climático suele producirse contaminación de las fuentes de agua o... https://t.co/fXsXx4jOwo
30000 Quería lluvia pero no tanto!!!
40000 Ja ja ja Ojalá Le Caiga Un Trueno Y Tengamos Una Falsa Menos😂😂
50000 Lluvia hermosa, pero nunca la luz no cortada
60000 alguien quiere ver la lluvia de estrellas conmigo? https://t.co/Y3qC1DM2Ex
70000 Quién estaba tan matada anoche que practicamente no sintio la tormenta?
80000 DIJERON QUE LA LLUVIA DE ESTRELLAS IBA A SER EL 14 O 15 Y FUE ANOCHE, LOS ODIO A TODO

## ENSERIO

In [85]:
data = pd.DataFrame({'word_ix':list(word_ix),'word_cant':list(word_cant.values())})


In [86]:
data.to_csv('diccionario.csv')

In [4]:

df = pd.read_csv('diccionario.csv')
word = df['word_ix'].values
word_ix = { i : j for j,i in enumerate(word) }

In [5]:
#Hago el bag of words

if os.path.exists('bow.txt')==False:
    bow = open('bow.txt','x')
else: bow = open('bow.txt','w')

    
    
for i,t in enumerate(msj): 
    
    bow_i = make_bow_vector(t,word_ix)
    for b in bow_i:
        for j in b:
            bow.write(str(j))
        bow.write(' ')
    if i % 10000==0:
        print(i,str(bow_i))
        time.sleep(1)



0 [[0 1 1 ... 0 0 0]]


KeyboardInterrupt: 

In [6]:
#b =  open('test.txt','x')
#b.close()
b = open('test.txt','w')
t = msj[1]
for i in make_bow_vector(t,word_ix): 
    for j in i:
        b.write(str(j))
        
    b.write(' ')
b.close()

b = open('test.txt','r')
a = b.read()
list((a))

['0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0'

In [130]:
bow.close()

In [116]:
a = make_bow_vector('las olas y el viento',word_ix)

In [14]:
bow =  open('bow.txt','r')
list(bow.readline(len(word_ix)))

['0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0'

In [8]:
list(bow.readline(len(word_ix)))

UnsupportedOperation: not readable

In [128]:
bow = open('bow.txt','r')
for i in range(10):
    print(bow.readline(58759*2-1))
    time.sleep(1)

0. 1. 1. ... 0. 0. 0.0. 1. 1. ... 0. 0. 0.0. 1. 0. ... 0. 0. 0.0. 0. 1. ... 0. 0. 0.0. 0. 1. ... 0. 0. 0.0. 1. 0. ... 0. 0. 0.0. 0. 1. ... 0. 0. 0.0. 1. 1. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 1. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 1. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 0. 0. 0.0. 0. 0. ... 

In [109]:
b = open('bb.txt','x')

In [110]:
for i in range(100):
    b.write(str(i)+'\n')
b.close()

In [112]:
b = open('bb.txt','r')
for i in range(100):
    print (b.readline(i))
    time.sleep(1)


0


1

2

3

4

5

6

7

8

9

10



KeyboardInterrupt: 

In [46]:
len(word_ix)

58754

In [37]:
list(word_ix.keys())[58753]

'n9'

In [130]:
dd  =list(d[1].keys())
dd = np.asarray(dd)
#print(np.dot(dd,bow[0][0,:]))